# Activity Related to NTDS.dit Domain Hash Retrieval
Detects suspicious commands that could be related to activity that uses volume shadow copy to steal and retrieve hashes from the NTDS.dit file remotely

## Rule Content
```
- title: Activity Related to NTDS.dit Domain Hash Retrieval
  id: b932b60f-fdda-4d53-8eda-a170c1d97bbd
  status: experimental
  description: Detects suspicious commands that could be related to activity that
    uses volume shadow copy to steal and retrieve hashes from the NTDS.dit file remotely
  author: Florian Roth, Michael Haag
  references:
  - https://www.swordshield.com/2015/07/getting-hashes-from-ntds-dit-file/
  - https://room362.com/post/2013/2013-06-10-volume-shadow-copy-ntdsdit-domain-hashes-remotely-part-1/
  - https://www.trustwave.com/Resources/SpiderLabs-Blog/Tutorial-for-NTDS-goodness-(VSSADMIN,-WMIS,-NTDS-dit,-SYSTEM)/
  - https://securingtomorrow.mcafee.com/mcafee-labs/new-teslacrypt-ransomware-arrives-via-spam/
  - https://dfironthemountain.wordpress.com/2018/12/06/locked-file-access-using-esentutl-exe/
  tags:
  - attack.credential_access
  - attack.t1003
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine:
      - vssadmin.exe Delete Shadows
      - 'vssadmin create shadow /for=C:'
      - copy \\?\GLOBALROOT\Device\\*\windows\ntds\ntds.dit
      - copy \\?\GLOBALROOT\Device\\*\config\SAM
      - 'vssadmin delete shadows /for=C:'
      - 'reg SAVE HKLM\SYSTEM '
      - esentutl.exe /y /vss *\ntds.dit*
    condition: selection
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Administrative activity
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(vssadmin.exe\ Delete\ Shadows OR vssadmin\ create\ shadow\ \/for\=C\: OR copy\ \\?\\GLOBALROOT\\Device\\*\\windows\\ntds\\ntds.dit OR copy\ \\?\\GLOBALROOT\\Device\\*\\config\\SAM OR vssadmin\ delete\ shadows\ \/for\=C\: OR reg\ SAVE\ HKLM\\SYSTEM\  OR esentutl.exe\ \/y\ \/vss\ *\\ntds.dit*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()